# Fraud case study



## Day 1: building a fraud model

## Day 2: building an app/dashboard

## Tips success

You will quickly run out of time:

*  Use CRISP-DM workflow to analyze data and build a model
*  Iterate quickly, test often, commit often
*  Build deadlines for your work so you stay on track
*  Should have a model by end of day 1
*  Start app once model is working

### CRISP-DM workflow

Follow the [CRISP-DM](https://en.wikipedia.org/wiki/Cross_Industry_Standard_Process_for_Data_Mining) steps:

1.  Business understanding
2.  Data understanding
3.  Data preparation
4.  Modeling
5.  Evaluation
6.  Deployment

# Introduction to case study: data & problem

Let's look at the data.  What format is the data in?  How do you extract it?

In [1]:
ls -lh data

total 67280
-rw-r--r--  1 jbud  staff    33M May 27 20:23 data.zip


Unzip the data so you can load it into Python

In [2]:
!unzip data/data.zip -d data

Archive:  data/data.zip
  inflating: data/data.json          


Initially, work with a subset at first in order to iterate quickly.  But, the file is one giant line of json:

In [3]:
!wc data/data.json

       0 8173151 250481668 data/data.json


Write a quick and dirty script to pull out the first 100 records so we can get code working quickly.

In [4]:
%%writefile subset_json.py
"""head_json.py - extract a couple records from a huge json file.

Syntax: python head_json.py < infile.json > outfile.json
"""

import sys

start_char = '{'
stop_char = '}'
n_records = 100
level_nesting = 0

while n_records != 0:
    ch = sys.stdin.read(1)
    sys.stdout.write(ch)
    if ch == start_char:
        level_nesting += 1
    if ch == stop_char:
        level_nesting -= 1
        if level_nesting == 0:
            n_records -= 1
sys.stdout.write(']')


Writing subset_json.py


In [5]:
!python subset_json.py < data/data.json > data/subset.json

In [54]:
import pandas as pd

df = pd.read_json('data/subset.json')

In [55]:
df.info

<bound method DataFrame.info of           acct_type  approx_payout_date  body_length  channels country  \
0   fraudster_event          1266062400         3852         5      US   
1           premium          1296720000         3499         0      US   
2           premium          1296172800         2601         8      US   
3           premium          1388966400        12347         6      IE   
4           premium          1297900800         2417        11      US   
..              ...                 ...          ...       ...     ...   
95          premium          1328070600         1314         6      US   
96          premium          1324098000         1483         6      US   
97          premium          1325844000         1860         5      US   
98          premium          1325836500         2219        11      US   
99          premium          1357466400        12479         8      US   

   currency  delivery_method  \
0       USD                0   
1       USD    

In [56]:
df.head().T

,0,1,2,3,4
acct_type,fraudster_event,premium,premium,premium,premium
approx_payout_date,1266062400,1296720000,1296172800,1388966400,1297900800
body_length,3852,3499,2601,12347,2417
channels,5,0,8,6,11
country,US,US,US,IE,US
currency,USD,USD,USD,EUR,USD
delivery_method,0,1,1,1,0
description,"<p><a href=""http://s432.photobucket.com/albums...","<p>Join us for a quick, one-night, community-b...","<h3><span class=""subcategory""><strong>Teacher ...","<p style=""margin-bottom: 1.3em; padding-bottom...",<p>Writers and filmmakers need to understand t...
email_domain,gmail.com,ruf.org,pvsd.k12.ca.us,irishtabletennis.com,artsandbusinesscouncil.org
event_created,1262739706,1293832670,1291090956,1360681570,1291994666


Some of the data is text (and HTML), which will require feature engineering:

* TF-IDF
* Feature hashing
* n-grams

etc.

You will also need to construct a target from `acct_type`.  Fraud events start with `fraud`.  How you define fraud depends on how you define the business problem.

In [12]:
df.acct_type.value_counts(dropna=False)

premium             90
fraudster            3
spammer_warn         3
fraudster_event      2
spammer_noinvite     1
spammer_limited      1
Name: acct_type, dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 44 columns):
acct_type             100 non-null object
approx_payout_date    100 non-null int64
body_length           100 non-null int64
channels              100 non-null int64
country               100 non-null object
currency              100 non-null object
delivery_method       100 non-null int64
description           100 non-null object
email_domain          100 non-null object
event_created         100 non-null int64
event_end             100 non-null int64
event_published       100 non-null int64
event_start           100 non-null int64
fb_published          100 non-null int64
gts                   100 non-null float64
has_analytics         100 non-null int64
has_header            90 non-null float64
has_logo              100 non-null int64
listed                100 non-null object
name                  100 non-null object
name_length           100 non-null int64
num_order             100

Is missing data a problem?  What are your options for handling missing data?

In [14]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
approx_payout_date,100.0,1.314867e+09,1.455055e+07,1.266062e+09,1.304711e+09,1.315984e+09,1.322812e+09,1.388966e+09
body_length,100.0,3.114250e+03,2.814620e+03,0.000000e+00,1.077500e+03,1.920500e+03,4.361000e+03,1.247900e+04
channels,100.0,7.500000e+00,3.195641e+00,0.000000e+00,5.750000e+00,8.000000e+00,1.100000e+01,1.300000e+01
delivery_method,100.0,4.600000e-01,6.100174e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,3.000000e+00
event_created,100.0,1.309823e+09,1.272655e+07,1.262740e+09,1.300009e+09,1.310730e+09,1.318629e+09,1.360682e+09
event_end,100.0,1.314436e+09,1.455050e+07,1.265630e+09,1.304279e+09,1.315552e+09,1.322380e+09,1.388534e+09
event_published,100.0,1.310264e+09,1.282619e+07,1.263110e+09,1.300033e+09,1.312086e+09,1.318995e+09,1.360683e+09
event_start,100.0,1.314038e+09,1.330579e+07,1.265594e+09,1.304267e+09,1.315433e+09,1.321645e+09,1.360703e+09
fb_published,100.0,2.000000e-01,4.020151e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
gts,100.0,2.223905e+03,5.168755e+03,0.000000e+00,9.998500e+01,5.595900e+02,1.451150e+03,3.896353e+04


In [15]:
df

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,...,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,fraudster_event,1266062400,3852,5,US,USD,0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,...,"[{'event_id': 527017, 'cost': 25.0, 'availabil...",36,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL
1,premium,1296720000,3499,0,US,USD,1,"<p>Join us for a quick, one-night, community-b...",ruf.org,1293832670,...,"[{'event_id': 786878, 'cost': 35.0, 'availabil...",149,1280942776,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC
2,premium,1296172800,2601,8,US,USD,1,"<h3><span class=""subcategory""><strong>Teacher ...",pvsd.k12.ca.us,1291090956,...,"[{'event_id': 787337, 'cost': 93.51, 'availabi...",214,1272559388,3,10100 Pioneer Blvd Suite 100,US,33.944201,-118.080419,Los Angeles County Office of Education,CA
3,premium,1388966400,12347,6,IE,EUR,1,"<p style=""margin-bottom: 1.3em; padding-bottom...",irishtabletennis.com,1360681570,...,"[{'event_id': 885645, 'cost': 25.0, 'availabil...",889,1283870102,3,,None,NaN,NaN,None,None
4,premium,1297900800,2417,11,US,USD,0,<p>Writers and filmmakers need to understand t...,artsandbusinesscouncil.org,1291994666,...,"[{'event_id': 1114349, 'cost': 150.0, 'availab...",35,1288984065,3,One Marina Park Drive,US,42.353848,-71.044276,Fish & Richardson,MA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,premium,1328070600,1314,6,US,USD,0,<h3>Jan 26 at 7pm</h3>\r\n<h4>Opening Night Ga...,nwfilmforum.org,1322877746,...,"[{'event_id': 2589534, 'cost': 38.0, 'availabi...",211,1304619530,3,1515 12th Avenue,US,47.614380,-122.317031,Northwest Film Forum,WA
96,premium,1324098000,1483,6,US,USD,0,"<p> </p>\r\n<div style=""text-align: -webkit-au...",richmondicezone.com,1322879399,...,"[{'event_id': 2589614, 'cost': 8.0, 'availabil...",682,1263930774,4,4350 Pouncey Tract Road,US,37.654891,-77.613550,SkateNation Plus,VA
97,premium,1325844000,1860,5,US,USD,1,"<p style=""text-align: center;""><span style=""fo...",kineticevents.com,1323477465,...,"[{'event_id': 2621588, 'cost': 20.0, 'availabi...",359,1292438475,1,138 Minna St.,US,37.786823,-122.400339,John Colins,CA
98,premium,1325836500,2219,11,US,USD,1,"<p style=""font-size: 10pt;"" align=""left""><img ...",cecnc.com,1325106030,...,"[{'event_id': 2687671, 'cost': 0.0, 'availabil...",919,1245718455,3,111 Minna St,US,37.787429,-122.399384,111 Minna Gallery,CA


In [21]:
df.columns

Index(['acct_type', 'approx_payout_date', 'body_length', 'channels', 'country',
       'currency', 'delivery_method', 'description', 'email_domain',
       'event_created', 'event_end', 'event_published', 'event_start',
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
       'listed', 'name', 'name_length', 'num_order', 'num_payouts',
       'object_id', 'org_desc', 'org_facebook', 'org_name', 'org_twitter',
       'payee_name', 'payout_type', 'previous_payouts', 'sale_duration',
       'sale_duration2', 'show_map', 'ticket_types', 'user_age',
       'user_created', 'user_type', 'venue_address', 'venue_country',
       'venue_latitude', 'venue_longitude', 'venue_name', 'venue_state'],
      dtype='object')

In [29]:
column_subset = ['acct_type','has_analytics', 'has_header', 'has_logo',
       'listed', 'name', 'name_length', 'num_order', 'num_payouts',
       'object_id', 'org_desc', 'org_facebook', 'org_name', 'org_twitter',
       'payee_name', 'payout_type']

In [32]:
df['acct_type'].unique()

array(['fraudster_event', 'premium', 'spammer_warn', 'fraudster',
       'spammer_limited', 'spammer_noinvite'], dtype=object)

In [46]:
df

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,...,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,True,1266062400,3852,5,US,USD,0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,...,"[{'event_id': 527017, 'cost': 25.0, 'availabil...",36,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL
1,False,1296720000,3499,0,US,USD,1,"<p>Join us for a quick, one-night, community-b...",ruf.org,1293832670,...,"[{'event_id': 786878, 'cost': 35.0, 'availabil...",149,1280942776,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC
2,False,1296172800,2601,8,US,USD,1,"<h3><span class=""subcategory""><strong>Teacher ...",pvsd.k12.ca.us,1291090956,...,"[{'event_id': 787337, 'cost': 93.51, 'availabi...",214,1272559388,3,10100 Pioneer Blvd Suite 100,US,33.944201,-118.080419,Los Angeles County Office of Education,CA
3,False,1388966400,12347,6,IE,EUR,1,"<p style=""margin-bottom: 1.3em; padding-bottom...",irishtabletennis.com,1360681570,...,"[{'event_id': 885645, 'cost': 25.0, 'availabil...",889,1283870102,3,,None,NaN,NaN,None,None
4,False,1297900800,2417,11,US,USD,0,<p>Writers and filmmakers need to understand t...,artsandbusinesscouncil.org,1291994666,...,"[{'event_id': 1114349, 'cost': 150.0, 'availab...",35,1288984065,3,One Marina Park Drive,US,42.353848,-71.044276,Fish & Richardson,MA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,1328070600,1314,6,US,USD,0,<h3>Jan 26 at 7pm</h3>\r\n<h4>Opening Night Ga...,nwfilmforum.org,1322877746,...,"[{'event_id': 2589534, 'cost': 38.0, 'availabi...",211,1304619530,3,1515 12th Avenue,US,47.614380,-122.317031,Northwest Film Forum,WA
96,False,1324098000,1483,6,US,USD,0,"<p> </p>\r\n<div style=""text-align: -webkit-au...",richmondicezone.com,1322879399,...,"[{'event_id': 2589614, 'cost': 8.0, 'availabil...",682,1263930774,4,4350 Pouncey Tract Road,US,37.654891,-77.613550,SkateNation Plus,VA
97,False,1325844000,1860,5,US,USD,1,"<p style=""text-align: center;""><span style=""fo...",kineticevents.com,1323477465,...,"[{'event_id': 2621588, 'cost': 20.0, 'availabi...",359,1292438475,1,138 Minna St.,US,37.786823,-122.400339,John Colins,CA
98,False,1325836500,2219,11,US,USD,1,"<p style=""font-size: 10pt;"" align=""left""><img ...",cecnc.com,1325106030,...,"[{'event_id': 2687671, 'cost': 0.0, 'availabil...",919,1245718455,3,111 Minna St,US,37.787429,-122.399384,111 Minna Gallery,CA


In [90]:
df = pd.read_json('data/subset.json')

In [91]:
#Payout date - relation to 

def is_fraud(df, col):
    '''
    Replaces fraud with 1 for true, 0 for false
    '''
    fraud_descriptors = ['fraudster_event', 'fraudster']
    df[col] = df[col].apply(lambda x: x in fraud_descriptors)
    df[col].replace(True, int(1), inplace = True)
    return df

In [92]:
is_fraud(df, 'acct_type')

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,...,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,1.0,1266062400,3852,5,US,USD,0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,...,"[{'event_id': 527017, 'cost': 25.0, 'availabil...",36,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL
1,0.0,1296720000,3499,0,US,USD,1,"<p>Join us for a quick, one-night, community-b...",ruf.org,1293832670,...,"[{'event_id': 786878, 'cost': 35.0, 'availabil...",149,1280942776,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC
2,0.0,1296172800,2601,8,US,USD,1,"<h3><span class=""subcategory""><strong>Teacher ...",pvsd.k12.ca.us,1291090956,...,"[{'event_id': 787337, 'cost': 93.51, 'availabi...",214,1272559388,3,10100 Pioneer Blvd Suite 100,US,33.944201,-118.080419,Los Angeles County Office of Education,CA
3,0.0,1388966400,12347,6,IE,EUR,1,"<p style=""margin-bottom: 1.3em; padding-bottom...",irishtabletennis.com,1360681570,...,"[{'event_id': 885645, 'cost': 25.0, 'availabil...",889,1283870102,3,,None,NaN,NaN,None,None
4,0.0,1297900800,2417,11,US,USD,0,<p>Writers and filmmakers need to understand t...,artsandbusinesscouncil.org,1291994666,...,"[{'event_id': 1114349, 'cost': 150.0, 'availab...",35,1288984065,3,One Marina Park Drive,US,42.353848,-71.044276,Fish & Richardson,MA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,1328070600,1314,6,US,USD,0,<h3>Jan 26 at 7pm</h3>\r\n<h4>Opening Night Ga...,nwfilmforum.org,1322877746,...,"[{'event_id': 2589534, 'cost': 38.0, 'availabi...",211,1304619530,3,1515 12th Avenue,US,47.614380,-122.317031,Northwest Film Forum,WA
96,0.0,1324098000,1483,6,US,USD,0,"<p> </p>\r\n<div style=""text-align: -webkit-au...",richmondicezone.com,1322879399,...,"[{'event_id': 2589614, 'cost': 8.0, 'availabil...",682,1263930774,4,4350 Pouncey Tract Road,US,37.654891,-77.613550,SkateNation Plus,VA
97,0.0,1325844000,1860,5,US,USD,1,"<p style=""text-align: center;""><span style=""fo...",kineticevents.com,1323477465,...,"[{'event_id': 2621588, 'cost': 20.0, 'availabi...",359,1292438475,1,138 Minna St.,US,37.786823,-122.400339,John Colins,CA
98,0.0,1325836500,2219,11,US,USD,1,"<p style=""font-size: 10pt;"" align=""left""><img ...",cecnc.com,1325106030,...,"[{'event_id': 2687671, 'cost': 0.0, 'availabil...",919,1245718455,3,111 Minna St,US,37.787429,-122.399384,111 Minna Gallery,CA


In [93]:
df['column_subset']

KeyError: 'column_subset'

In [87]:
#make sure have values in test data - sam sent out link
'''
'has_analytics' - yes
'has_header' - yes
'has_logo' - yes
'listed' - yes
'name' - yes
'name_length' - yes
'num_order' - yes
'num_payouts' - yes
'object_id' - yes
'org_desc' - yes, but empty
'org_facebook' - yes
'org_name' - yes
'org_twitter' - yes
'payee_name' - yes, but empty 
'payout_type' - yes
'''
print('test')

test
